# V1 to V2 Conversion + Finetuning

This notebook:
1. Loads a trained V1 checkpoint
2. Converts V1 scales to V2 format (unit-norm + rank_magnitude)
3. Finetunes the V2 model with scale-only training

## V1 → V2 Conversion:
- V1: `scales = scale_A @ scale_B` (arbitrary magnitudes)
- V2: `scales = (g * A_dir) @ B_dir` where A_dir, B_dir are unit-norm

The conversion extracts norms from V1 scales and stores them in `rank_magnitude`.

In [ ]:
# ============================================================
# GOOGLE DRIVE PATHS
# ============================================================

GD_RUNS = '/content/drive/MyDrive/qwen3_runs'
GD_CACHES = '/content/drive/MyDrive/qwen3_caches'

LOCAL_RUNS = 'runs'
LOCAL_CACHES = 'caches'

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Clone/update repo
!git clone https://github.com/anemll/qwen3_apple_style_2bit_qat_lora.git || (cd qwen3_apple_style_2bit_qat_lora && git pull)
%cd qwen3_apple_style_2bit_qat_lora
!git fetch && git pull && git reset --hard HEAD

import sys
[sys.modules.pop(k) for k in list(sys.modules) if k.startswith('qat_lora')]

# Install dependencies
!pip install -q transformers accelerate safetensors

In [ ]:
# ============================================================
# CONFIGURATION
# ============================================================

import torch
import os

MODEL_ID = 'Qwen/Qwen3-0.6B'

# V1 checkpoint archive name (the .tgz file in GD_RUNS)
V1_ARCHIVE = 'anemll_q4_a4_e2e_v2_scales_only.tgz'
V1_FOLDER = 'anemll_q4_a4_e2e_v2_scales_only'

# Extract V1 checkpoint from .tgz
os.makedirs(LOCAL_RUNS, exist_ok=True)
v1_extract_path = f'{LOCAL_RUNS}/{V1_FOLDER}'
if not os.path.exists(v1_extract_path):
    print(f'Extracting {V1_ARCHIVE} from Google Drive...')
    !tar -xzf {GD_RUNS}/{V1_ARCHIVE} -C {LOCAL_RUNS}/
else:
    print(f'V1 checkpoint already extracted at {v1_extract_path}')

# Path to the actual checkpoint file
V1_CHECKPOINT = f'{v1_extract_path}/model_state_dict.pt'
print(f'V1 checkpoint: {V1_CHECKPOINT}')

# Verify it exists
assert os.path.exists(V1_CHECKPOINT), f'V1 checkpoint not found at {V1_CHECKPOINT}'

# Quantization config (must match V1 checkpoint)
LUT_BITS = 4
LUT_SIZE = 2**LUT_BITS
SCALE_RANK = 4

ATTN_LUT_BITS = 4
ATTN_LUT_SIZE = 2**ATTN_LUT_BITS
ATTN_SCALE_RANK = 4

# Training
BATCH_SIZE = 32 if torch.cuda.is_available() else 4
DISTILL_TEMP = 2.0

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DTYPE = torch.bfloat16

QUAL = f'q{LUT_BITS}_a{ATTN_LUT_BITS}'
print(f'Quality: {QUAL}')
print(f'Device: {DEVICE}, dtype: {DTYPE}')

In [ ]:
# ============================================================
# LOAD KD CACHE
# ============================================================

import os

CACHE_NAME = 'alpaca_chat_think_both_L128_K128_R1024'
CACHE_TGZ = f'{CACHE_NAME}.tgz'

!mkdir -p {LOCAL_CACHES}

cache_local_path = f'{LOCAL_CACHES}/{CACHE_NAME}'
if not os.path.exists(cache_local_path):
    print(f'Extracting {CACHE_TGZ} from Google Drive...')
    !tar -xzf {GD_CACHES}/{CACHE_TGZ} -C {LOCAL_CACHES}/
else:
    print(f'Cache already exists at {cache_local_path}')

!ls -la {cache_local_path}/ | head -5

# Step 1: Load V1 Model and Checkpoint

In [ ]:
# ============================================================
# LOAD BASE MODEL
# ============================================================

from transformers import AutoModelForCausalLM, AutoTokenizer

print(f'Loading {MODEL_ID}...')
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)

# Load for V1
v1_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=DTYPE,
    trust_remote_code=True,
)
print(f'Base model loaded: {sum(p.numel() for p in v1_model.parameters()):,} params')

In [ ]:
# ============================================================
# REPLACE WITH V1 LAYERS AND LOAD CHECKPOINT
# ============================================================

import sys
sys.path.insert(0, '.')

import importlib
import qat_lora
importlib.reload(qat_lora)

from qat_lora import (
    AnemllQATLinear,
    AnemllQuantConfig,
    replace_linear_with_anemll,
    load_checkpoint,
)

# Create V1 configs
v1_mlp_config = AnemllQuantConfig(
    lut_size=LUT_SIZE,
    scale_rank=SCALE_RANK,
)
v1_attn_config = AnemllQuantConfig(
    lut_size=ATTN_LUT_SIZE,
    scale_rank=ATTN_SCALE_RANK,
)

# Replace with V1 layers
print('Replacing with V1 AnemllQATLinear...')
replace_linear_with_anemll(
    v1_model,
    config=v1_mlp_config,
    attn_config=v1_attn_config,
)

# Load V1 checkpoint
print(f'\nLoading V1 checkpoint from {V1_CHECKPOINT}...')
v1_model.load_state_dict(torch.load(V1_CHECKPOINT, map_location='cpu'), strict=False)
v1_model.to(DEVICE)
print('V1 checkpoint loaded!')

In [ ]:
# ============================================================
# EVALUATE V1 MODEL
# ============================================================

from qat_lora import evaluate_kd_loss

v1_model.eval()
v1_loss = evaluate_kd_loss(v1_model, cache_local_path, DEVICE, num_samples=50, temperature=DISTILL_TEMP)
print(f'V1 KD Loss: {v1_loss:.4f}')

# Step 2: Create V2 Model and Convert

In [ ]:
# ============================================================
# CREATE V2 MODEL
# ============================================================

from qat_lora import (
    AnemllQATLinearV2,
    AnemllQuantConfigV2,
    replace_linear_with_anemll_v2,
    freeze_Q_all,
)

# Load fresh base model for V2
print('Loading fresh base model for V2...')
v2_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=DTYPE,
    trust_remote_code=True,
)

# Create V2 configs
v2_mlp_config = AnemllQuantConfigV2(
    lut_size=LUT_SIZE,
    scale_rank=SCALE_RANK,
)
v2_attn_config = AnemllQuantConfigV2(
    lut_size=ATTN_LUT_SIZE,
    scale_rank=ATTN_SCALE_RANK,
)

# Replace with V2 layers
print('Replacing with V2 AnemllQATLinearV2...')
replace_linear_with_anemll_v2(
    v2_model,
    mlp_config=v2_mlp_config,
    attn_config=v2_attn_config,
)

v2_model.to(DEVICE)
print('V2 model created!')

In [ ]:
# ============================================================
# CONVERT V1 → V2
# ============================================================

def convert_v1_layer_to_v2(v1_layer, v2_layer):
    """Convert V1 layer parameters to V2 format.
    
    V1: scales = A @ B (arbitrary magnitudes)
    V2: scales = (g * A_dir) @ B_dir (unit-norm + magnitude)
    """
    with torch.no_grad():
        # Copy base parameters
        v2_layer.weight.data = v1_layer.weight.data.clone()
        if v1_layer.bias is not None and v2_layer.bias is not None:
            v2_layer.bias.data = v1_layer.bias.data.clone()
        v2_layer.lut.data = v1_layer.lut.data.clone()
        
        # Get V1 scales (handle potential padding)
        A = v1_layer.scale_A  # [out, rank]
        B_full = v1_layer.scale_B  # [rank, padded_in]
        B = B_full[:, :v1_layer.in_features]  # [rank, in]
        
        # Compute norms
        A_norms = A.norm(dim=0, keepdim=True).clamp(min=1e-8)  # [1, rank]
        B_norms = B.norm(dim=1, keepdim=True).clamp(min=1e-8)  # [rank, 1]
        
        # V2 stores unit-norm directions + magnitude
        A_dir = A / A_norms  # [out, rank] unit-norm columns
        B_dir = B / B_norms  # [rank, in] unit-norm rows
        
        # Magnitude is product of norms
        rank_magnitude = (A_norms.squeeze() * B_norms.squeeze())  # [rank]
        
        # Store in V2 layer
        v2_layer.scale_A.data = A_dir.to(v1_layer.weight.dtype)
        v2_layer.scale_B.data = B_dir.to(v1_layer.weight.dtype)
        v2_layer.rank_magnitude.data = rank_magnitude.to(v1_layer.weight.dtype)


print('Converting V1 → V2...')
converted = 0

# Collect V1 and V2 layers
v1_layers = {name: m for name, m in v1_model.named_modules() 
             if type(m).__name__ == 'AnemllQATLinear'}
v2_layers = {name: m for name, m in v2_model.named_modules() 
             if type(m).__name__ == 'AnemllQATLinearV2'}

print(f'Found {len(v1_layers)} V1 layers, {len(v2_layers)} V2 layers')

# Convert each layer
for name in v1_layers:
    if name in v2_layers:
        convert_v1_layer_to_v2(v1_layers[name], v2_layers[name])
        converted += 1
        if converted <= 3:
            print(f'  Converted: {name}')

print(f'\nConverted {converted} layers')

# Free V1 model memory
del v1_model
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# ============================================================
# FREEZE Q FOR V2
# ============================================================

print('Freezing Q (computing indices once)...')
freeze_Q_all(v2_model, verbose=False)
print('Q frozen for all V2 layers.')

In [ ]:
# ============================================================
# EVALUATE CONVERTED V2 MODEL
# ============================================================

v2_model.eval()
v2_converted_loss = evaluate_kd_loss(v2_model, cache_local_path, DEVICE, num_samples=50, temperature=DISTILL_TEMP)
print(f'\n=== Conversion Results ===')
print(f'V1 Loss: {v1_loss:.4f}')
print(f'V2 Loss (after conversion): {v2_converted_loss:.4f}')
print(f'Difference: {abs(v2_converted_loss - v1_loss):.4f}')

if abs(v2_converted_loss - v1_loss) < 0.1:
    print('Conversion successful - losses are close!')
else:
    print('Note: Some difference expected due to different forward implementations')

# Step 3: Finetune V2 Model

In [ ]:
# ============================================================
# V2 SCALE FINETUNING (MLP + ATTENTION)
# ============================================================

from qat_lora import train_e2e, unfreeze_model_for_training_v2

# Unfreeze scales for training
unfreeze_model_for_training_v2(v2_model)

# Enable all scales
for name, module in v2_model.named_modules():
    if type(module).__name__ == 'AnemllQATLinearV2':
        if hasattr(module, 'scale_A') and module.scale_A is not None:
            module.scale_A.requires_grad = True
            module.scale_B.requires_grad = True
            module.rank_magnitude.requires_grad = True
        module.weight.requires_grad = False  # Keep weights frozen

trainable = sum(p.numel() for p in v2_model.parameters() if p.requires_grad)
print(f'Trainable params: {trainable:,}')

# Train all scales
print('\nStarting V2 scale finetuning...')
e2e_result = train_e2e(
    model=v2_model,
    cache_dir=cache_local_path,
    device=DEVICE,
    max_steps=2000,
    batch_size=BATCH_SIZE,
    lr=1e-4,
    use_cosine_schedule=True,
    warmup_steps=100,
    min_lr_ratio=0.1,
    temperature=DISTILL_TEMP,
    train_weights=False,
    train_scales=True,
    hard_top1_weight=0.0,
    hard_full_weight=0.0,
    logging_steps=20,
    eval_steps=100,
    verbose=True,
    train_mlp_only=False,  # Train all scales
)

In [ ]:
# ============================================================
# SAVE CONVERTED + FINETUNED V2 CHECKPOINT
# ============================================================

import os

V2_RUN_NAME = f'anemll_v2_{QUAL}_from_v1_finetuned'
V2_SAVE_DIR = f'{LOCAL_RUNS}/{V2_RUN_NAME}'
os.makedirs(V2_SAVE_DIR, exist_ok=True)

# Save checkpoint
torch.save(v2_model.state_dict(), f'{V2_SAVE_DIR}/model_state_dict.pt')

# Also save to tmp for quick access
torch.save(v2_model.state_dict(), '/tmp/v2_from_v1_finetuned.pt')

print(f'Saved V2 checkpoint to {V2_SAVE_DIR}')
print(f'Also saved to /tmp/v2_from_v1_finetuned.pt')

In [ ]:
# ============================================================
# FINAL EVALUATION
# ============================================================

v2_model.eval()
final_loss = evaluate_kd_loss(v2_model, cache_local_path, DEVICE, num_samples=50, temperature=DISTILL_TEMP)

print(f'\n=== Final Results ===')
print(f'V1 Original:        {v1_loss:.4f}')
print(f'V2 After Convert:   {v2_converted_loss:.4f}')
print(f'V2 After Finetune:  {final_loss:.4f}')
print(f'Total Improvement:  {v1_loss - final_loss:.4f}')

In [ ]:
# ============================================================
# UPLOAD TO GOOGLE DRIVE
# ============================================================

!tar -czvf {V2_RUN_NAME}.tgz -C {LOCAL_RUNS} {V2_RUN_NAME}
!cp {V2_RUN_NAME}.tgz {GD_RUNS}/
print(f'\nUploaded to {GD_RUNS}/{V2_RUN_NAME}.tgz')

# Test Inference

In [ ]:
# ============================================================
# FREEZE FOR INFERENCE
# ============================================================

from qat_lora import freeze_model_for_inference_v2

print('Freezing V2 model for inference...')
freeze_model_for_inference_v2(v2_model, verbose=False)
print('Ready for inference!')

In [ ]:
# ============================================================
# TEST INFERENCE
# ============================================================

def run_inference(model, tokenizer, prompt, max_new_tokens=256):
    messages = [
        {'role': 'system', 'content': 'You are a helpful assistant.'},
        {'role': 'user', 'content': prompt}
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(text, return_tensors='pt').to(DEVICE)
    
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False)
    
    return tokenizer.decode(output[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)

prompts = [
    'What is the capital of France?',
    'Explain quantum mechanics briefly.',
    'What is 2+2?',
]

v2_model.eval()
for prompt in prompts:
    response = run_inference(v2_model, tokenizer, prompt)
    print(f'Prompt: {prompt}')
    print(f'Response: {response}')
    print('-' * 50)